# U-Net

> Neural net model

In [ ]:
#| default_exp models.unet

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
#| export
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch_lr_finder import LRFinder

from omegaconf import OmegaConf
from hydra.utils import instantiate

from matplotlib import pyplot as plt

from nimrod.models.conv import ConvLayer
from nimrod.models.resnet import ResBlock
from nimrod.utils import get_device, set_seed

from typing import List
import logging

/user/s/slegroux/miniconda3/envs/nimrod/lib/python3.11/site-packages/torch_lr_finder/lr_finder.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
Seed set to 42
Seed set to 42


In [ ]:
#| export
logger = logging.getLogger(__name__)
set_seed()

Seed set to 42


## Tiny Unet

In [ ]:
def up_block(ni, nf, kernel_size=3, norm=None):
    return nn.Sequential(
        nn.UpsamplingNearest2d(scale_factor=2),
        ResBlock(ni, nf, kernel_size=kernel_size)
    )

In [ ]:
x = torch.randn(1, 3, 128, 128)
up_block(3, 6)(x).shape

[16:47:36] WARNING - setting conv bias to False as Batchnorm is used
[16:47:36] WARNING - setting conv bias to False as Batchnorm is used
[16:47:36] WARNING - setting conv bias to False as Batchnorm is used


torch.Size([1, 6, 256, 256])

In [ ]:
class TinyUnet(nn.Module):
    def __init__(
            self,
            in_channels: int = 3,
            n_features: List[int]= [32, 64, 128, 256, 512, 1024],
            normalization=nn.BatchNorm2d,
            activation=nn.ReLU,
            ):
        
        super().__init__()

        self.layers = nn.ModuleList()
        self.layers.append(nn.Identity())
        start = ResBlock(in_channels, n_features[0])
        dn = nn.ModuleList(
            [ResBlock(n_features[i], n_features[i+1]) for i in range(len(n_features)-1)]
            )
        self.layers += dn

        self.up = nn.ModuleList(
            [up_block(n_features[i], n_features[i-1]) for i in range(len(n_features)-1,0,-1)]
            )
        
        self.up += [ResBlock(n_features[0], 3)]
        self.end = ResBlock(3, 3)

    def forward(self, x):
        n = len(self.layers)
        for i,l in enumerate(self.up):
            if i!=0: x += self.layers[n-i]
            x = l(x)
        return self.end(x+self.layers[0])

        
        # x = self.start(x)
        # for l in self.dn:
        #     layers.append(x)
        #     x = l(x)
        # n = len(layers)
        # for i,l in enumerate(self.up):
        #     if i!=0: x += layers[n-i]
        #     x = l(x)
        # return self.end(x+layers[0])
        



        


In [ ]:
model = TinyUnet()
x = torch.randn(1, 3, 128, 128)
model(x).shape

[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv bias to False as Batchnorm is used
[16:53:36] WARNING - setting conv 

NotImplementedError: Module [TinyUnet] is missing the required "forward" function

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()